## Goal: understanding contribution patterns

Analyzing groups of contributors, according to their activity patterns, and their evolution over time, helps to understand the structure of the community. These groups will be defined according to how much active they are (from casual to core contributors), and which kinds of activity they have (for example, producing code, reviewing code, submitting issues, contributing in discussions, etc.). Whenever convenient, the characterization will be combined with the contributor groups identified in the first goal.

This goal is refined in the following questions:

**Questions**:

 * How often do contributors contribute?
 * How is the structure of contribution, according to level of activity?
 * How is the structure of contribution, according to the different data sources?
 * How are the structures of contribution evolving over time?
 * How is people flowing in the structure of contribution?

These questions can be answered with the following metrics:

**Metrics**:

(Still to be refined)

 * Groups of contributors, by level of activity (core, regular, casual)
 * Groups of contributors, by kind of activity (committing, opening issues, merging pull requests, etc.)
 * Groups of contributors, by kind of activity (specialists, spread, etc.)
 * Activity metrics for each group
 * Absolute number of contributors moving from one group to another
 * Fraction of contributors moving from a group to another
 
Some of these metrics will be computed for the speficied contributor groups, over time.

# Metric Calculations
First we need to load a connection against the proper ES instance. We use an external module to load credentials from a file that will not be shared. If you want to run this, please use your own credentials, just put them in a file named '.settings' (in the same directory as this notebook) following the example file 'settings.sample'.

This section includes common code to manage and plot data. Queries will be available at the corresponding section.


In [1]:
import pandas

import os

import plotly as plotly
import plotly.graph_objs as go

import util as ut

from util import ESConnection
from elasticsearch_dsl import Search

es_conn = ESConnection()

# Let's load projects from the REVIEWED SPREADSHEET
projects = ut.read_projects("data/Contributors and Communities Analysis - Project grouping.xlsx")

project_name = os.environ.get('PROJECT', 'all')

date_range = {'gte': '2010-01-01', 'lt': 'now/y'}


In [2]:
def create_search(source):
    s = Search(using=es_conn, index=source)
    
    if source == 'git' or source == 'github':
        github = projects['Github']
        repos = github['Repo'].tolist()
        #print (repos)
        s = s.filter('terms', repo_name=repos)
    
    # TODO: Add bot and merges filtering.
    #s = s.filter('range', grimoire_creation_date={'gt': 'now/M-2y', 'lt': 'now/M'})
    return s

In [3]:
def print_df(result, group_field, value_field, group_column, value_column):
    df = pandas.DataFrame()

    df = df.from_dict(result.to_dict()['aggregations'][group_field]['buckets'])
    df = df.drop('doc_count', axis=1)
    df[value_field] = df[value_field].apply(lambda row: row['value'])
    df=df[['key', value_field]]
    df.columns = [group_column, value_column]

    return df

In [4]:
def stack_by(result, group_column, time_column, value_column, group_field, time_field, value_field):
    """Creates a dataframe based on group and time values
    """
    df = pandas.DataFrame(columns=[group_column, time_column, value_column])

    for b in result.to_dict()['aggregations'][group_field]['buckets']:
        for i in b[time_field]['buckets']:
            df.loc[len(df)] = [b['key'], i['key_as_string'], i[value_field]['value']]
    
    return df

def stack_by_2(result, group_column, time_column, value_column, group_field, time_field, value_field):
    """Creates a dataframe based on group and time values
    """
    df = pandas.DataFrame(columns=[time_column, group_column, value_column])

    for b in result.to_dict()['aggregations'][time_field]['buckets']:
        for i in b[group_field]['buckets']:
            df.loc[len(df)] = [b['key_as_string'], i['key'], i[value_field]['value']]
    
    return df

In [5]:
def onion(df, bucket_column, time_column, value_column):
    
    total = df[value_column].sum()
    
    percent_80 = total * 0.8
    percent_95 = total * 0.95
    core = 0
    core_sum = 0
    regular = 0
    regular_sum = 0
    casual = 0
    core_non = 0
    regular_non = 0
    casual_non = 0
    core_emp = 0
    regular_emp = 0
    casual_emp = 0

    for row in df.iterrows():
        value = row[1][value_column]
        non = False
        if row[1]['org'] == 'Non-Employees':
            non = True
        
        if (percent_80 > core_sum):
            core = core + 1
            core_sum = core_sum + value
            regular_sum = regular_sum + value
            if non:
                core_non = core_non + 1
            else:
                core_emp = core_emp + 1
                
        elif percent_95 > regular_sum:
            regular = regular + 1
            regular_sum = regular_sum + value
            if non:
                regular_non = regular_non + 1
            else:
                regular_emp = regular_emp + 1
        else:
            casual = casual + 1
            if non:
                casual_non = casual_non + 1
            else:
                casual_emp = casual_emp + 1

    return {"core":core,
            "regular":regular,
            "casual":casual,
            "core-non": core_non,
            "regular-non": regular_non,
            "casual-non": casual_non,
            "core-emp": core_emp,
            "regular-emp": regular_emp,
            "casual-emp": casual_emp} 

def onion_evolution(df, bucket_field, time_field, metric_field):
    
    #print(len(df))
    
    onion_df = pandas.DataFrame(
        columns=['Time', 
                 'Core', 'Core-Non', 'Core-Emp',
                 'Regular', 'Regular-Non', 'Regular-Emp',
                 'Casual', 'Casual-Non', 'Casual-Emp'])
    
    for time in df[time_field].unique():
        slice_df = df.loc[df['time'] == time]
        slice_df = slice_df.sort_values(by=metric_field, ascending=False)
        onion_result = onion(slice_df, 
                             bucket_column=bucket_field, 
                             time_column=time_field,
                             value_column=metric_field)
        #print(time, '->', len(slice_df))#, slice_df.columns.values.tolist(), '->', onion_result)
        onion_df.loc[len(onion_df)] = [time, 
                                       onion_result['core'],
                                       onion_result['core-non'], 
                                       onion_result['core-emp'],
                                       onion_result['regular'],
                                       onion_result['regular-non'],
                                       onion_result['regular-emp'],
                                       onion_result['casual'], 
                                       onion_result['casual-non'],
                                       onion_result['casual-emp']]
        
    
    #print(len(df))
    return onion_df

In [6]:
def print_grouped_bar(df, x_column, value_columns, title):
    """
    """
    plotly.offline.init_notebook_mode(connected=True)

    bars = []
    x_values = df[x_column].tolist()
    for value_column in value_columns:
        bars.append(go.Bar(
            x=x_values,
            y=df[value_column].tolist(),
            name=value_column))

    layout = go.Layout(
        barmode='group',
        title= title
    )

    fig = go.Figure(data=bars, layout=layout)
    plotly.offline.iplot(fig, filename='grouped-bar')
    
def print_stacked_bar(df, x_column, value_columns, title):
    """
    """
    plotly.offline.init_notebook_mode(connected=True)

    bars = []
    x_values = df[x_column].tolist()
    for value_column in value_columns:
        bars.append(go.Bar(
            x=x_values,
            y=df[value_column].tolist(),
            name=value_column))

    layout = go.Layout(
        barmode='stack',
        title= title
    )

    fig = go.Figure(data=bars, layout=layout)
    plotly.offline.iplot(fig, filename='stacked-bar')

In [7]:
def add_bot_filter(s):
    return s.filter('term', author_bot='false')

def add_merges_filter(s):
    return s.filter('range', files={'gt': 0})

def add_date_filter(s):
    return s.filter('range', grimoire_creation_date=date_range)

def add_general_date_filters(s):
    # 01/01/1998
    initial_ts = '883609200000'
    return s.filter('range', grimoire_creation_date={'gt': initial_ts})


def add_project_filter(s, project_name):

    if project_name.lower() != 'all':
        s = s.filter('term', project=project_name)

    return s

    # Let's load projects from the REVIEWED SPREADSHEET
    #projects = get_projects()
    # 
    # if project_name.lower() != 'all':
    #     github = projects['Github']
    #     repos = github[github['Project'] == project_name]['Repo'].tolist()
    #     #print(repos)
    #     s = s.filter('terms', repo_name=repos)
    # return s

# Let's load projects from the REVIEWED SPREADSHEET
#projects = ut.read_projects("data/Contributors and Communities Analysis - Project grouping.xlsx")


# Metrics

## Groups of contributors, by level of activity: core, regular, casual

Following table and chart shows number of contributors in three groups:
* Core: minimum number of authors who made 80% of contributions.
* Regular: minimum number of authors who made between 80% and 95% of contributions.
* Casual: the rest of contributors, who made the last 5% of contributions.

Looking at their evolution through time we can see the structure of a community at some point and its evolution.

In [8]:
s = create_search(source='git')

s = add_general_date_filters(s)

s = add_bot_filter(s)
s = add_merges_filter(s)

# Filter commits to the Project Repos
s = add_project_filter(s, project_name)

# Adds date range to retrieve data from
s = add_date_filter(s)


# Unique count of Commits by Authors over time
s.aggs.bucket('time', 'date_histogram', field='grimoire_creation_date', interval='quarter')\
    .bucket('uuid', 'terms', field='author_uuid', size=100000)\
    .metric('commits', 'cardinality', field='hash', precision_threshold=3000)
result = s.execute()

authors_df = stack_by_2(result, 'uuid', 'time', 'commits', 'uuid', 'time', 'commits')


s = create_search(source='git')

s = add_general_date_filters(s)

s = add_bot_filter(s)
s = add_merges_filter(s)

# Filter commits to the Project Repos
s = add_project_filter(s, project_name)

# Adds date range to retrieve data from
s = add_date_filter(s)


# Unique count of Commits by Authors over time
s.aggs.bucket('uuid', 'terms', field='author_uuid', size=100000)\
    .bucket('org', 'terms', field='author_org_name', size=1000)
result = s.execute()


orgs_df = pandas.DataFrame(columns=['uuid', 'org'])

for uuid in result.to_dict()['aggregations']['uuid']['buckets']:
    for org in uuid['org']['buckets']:
        orgs_df.loc[len(orgs_df)] = [uuid['key'], org['key']]
        
# Divide authors in Employees and Non-Employees based on org name
orgs_df.loc[orgs_df['org'] == 'Community', 'org'] = 'Non-Employees'
orgs_df.loc[orgs_df['org'] == 'Mozilla Staff', 'org'] = 'Employees'
orgs_df.loc[orgs_df['org'] == 'Code Sheriff', 'org'] = 'Employees'
# Add org to commits by author over time dataframe
authors_org_df = authors_df.merge(orgs_df, on='uuid', how='left')


In [9]:
onion_df=onion_evolution(authors_org_df, bucket_field='uuid', time_field='time', metric_field='commits')

# Calculate quarters
#onion_df['Quarter'] = pandas.PeriodIndex(pandas.to_datetime(onion_df.Time), freq='Q')
onion_df['Quarter'] = onion_df['Time'].map(lambda x: str(pandas.Period(x,'Q')))

onion_df

,Time,Core,Core-Non,Core-Emp,Regular,Regular-Non,Regular-Emp,Casual,Casual-Non,Casual-Emp,Quarter
0,2010-01-01T00:00:00.000Z,112.0,0.0,112.0,130.0,0.0,130.0,319.0,0.0,319.0,2010Q1
1,2010-04-01T00:00:00.000Z,125.0,0.0,125.0,140.0,0.0,140.0,373.0,0.0,373.0,2010Q2
2,2010-07-01T00:00:00.000Z,152.0,0.0,152.0,151.0,0.0,151.0,428.0,0.0,428.0,2010Q3
3,2010-10-01T00:00:00.000Z,159.0,0.0,159.0,139.0,0.0,139.0,373.0,0.0,373.0,2010Q4
4,2011-01-01T00:00:00.000Z,179.0,0.0,179.0,169.0,0.0,169.0,427.0,0.0,427.0,2011Q1
5,2011-04-01T00:00:00.000Z,209.0,0.0,209.0,194.0,0.0,194.0,455.0,0.0,455.0,2011Q2
6,2011-07-01T00:00:00.000Z,223.0,0.0,223.0,214.0,0.0,214.0,500.0,0.0,500.0,2011Q3
7,2011-10-01T00:00:00.000Z,236.0,0.0,236.0,197.0,0.0,197.0,523.0,0.0,523.0,2011Q4
8,2012-01-01T00:00:00.000Z,243.0,0.0,243.0,229.0,0.0,229.0,604.0,0.0,604.0,2012Q1
9,2012-04-01T00:00:00.000Z,287.0,0.0,287.0,249.0,0.0,249.0,628.0,0.0,628.0,2012Q2


In [10]:
print_grouped_bar(onion_df, 
                  'Quarter',
                  ['Core', 'Core-Non', 'Regular', 'Regular-Non', 'Casual', 'Casual-Non'],
                  'Contribution Groups: all developers / non-employees')
print_stacked_bar(onion_df, 
                  'Quarter',
                  ['Core-Emp', 'Core-Non', 'Regular-Emp', 'Regular-Non','Casual-Emp', 'Casual-Non'],
                  'Contribution Groups: employees / non-employees')

## Authors contributing to N projects

In [11]:
#results = []
#for i in analyzed_range:

# Buckets by author name, finding first commit for each of them
s = create_search(source='git')

# General filters
s = add_general_date_filters(s)
s = add_bot_filter(s)
s = add_merges_filter(s)

# Filter commits to the Project Repos
s = add_project_filter(s, project_name)


# Retrieve commits before given year
s = s.filter('range', grimoire_creation_date={'lt': 'now/y'})

# Bucketize by uuid and get first and last commit
s.aggs.bucket('authors', 'terms', field='author_uuid', size=100000) \
    .metric('projects', 'cardinality', field='project', precision_threshold=10000)
#print(s.to_dict())
result = s.execute()

project_count_df = print_df(result, group_field='authors', group_column='authors', value_field='projects',
                            value_column='projects')

project_count_df = project_count_df.groupby(['projects']).agg({'authors': 'count'})
#projects_df.rename(columns={"author": "# authors"}, inplace=True)
#projects_df = projects_df.reset_index().sort_values(by=['first_commit', '# authors'], ascending=[False, False])
project_count_df

plotly.offline.init_notebook_mode(connected=True)

bars = []

bars.append(go.Bar(
    x=project_count_df.reset_index()['projects'].tolist(),
    y=project_count_df['authors'].tolist()))

layout = go.Layout(
    barmode='bar'
)

fig = go.Figure(data=bars, layout=layout)
plotly.offline.iplot(fig, filename='authors-contributing-to-projects')


## Non-employees contributing to N projects

In [12]:
#results = []
#for i in analyzed_range:

# Buckets by author name, finding first commit for each of them
s = create_search(source='git')

# General filters
s = add_general_date_filters(s)
s = add_bot_filter(s)
s = add_merges_filter(s)

# Filter commits to the Project Repos
s = add_project_filter(s, project_name)


# Retrieve commits before given year
s = s.filter('range', grimoire_creation_date={'lt': 'now/y'})

# EXCLUDE EMPLOYEES
s = s.exclude('terms', author_org_name=['Mozilla Staff', 'Code Sheriff'])

# Bucketize by uuid and get first and last commit
s.aggs.bucket('authors', 'terms', field='author_uuid', size=100000) \
    .metric('projects', 'cardinality', field='project', precision_threshold=10000)
#print(s.to_dict())
result = s.execute()

project_count_df = print_df(result, group_field='authors', group_column='authors', value_field='projects', 
                            value_column='projects')

project_count_df = project_count_df.groupby(['projects']).agg({'authors': 'count'})
#projects_df.rename(columns={"author": "# authors"}, inplace=True)
#projects_df = projects_df.reset_index().sort_values(by=['first_commit', '# authors'], ascending=[False, False])
project_count_df

plotly.offline.init_notebook_mode(connected=True)

bars = []

bars.append(go.Bar(
    x=project_count_df.reset_index()['projects'].tolist(),
    y=project_count_df['authors'].tolist()))

layout = go.Layout(
    barmode='bar'
)

fig = go.Figure(data=bars, layout=layout)
plotly.offline.iplot(fig, filename='authors-contributing-to-projects')


## Authors across top 6 projects in 2016

In [13]:
 def get_project_intersection(df):
    project_project_df = pandas.DataFrame(columns=['project'])
    project_project_df.set_index('project')

    project_list = df['project'].unique()
    for project_row in project_list:
        row = {'project': project_row}
        for project_col in project_list:
            # compute intersection between projects
            authors_row_df = df.loc[df['project'] == project_row]['author']
            authors_col_df = df.loc[df['project'] == project_col]['author']
            intersection = len(set(authors_row_df).intersection(set(authors_col_df)))
            # Add intersection to row
            row[project_col] = intersection

        # add row to dataframe
        project_project_df = project_project_df.append(row, ignore_index=True)

    return project_project_df
    

In [14]:
#results = []
#for i in analyzed_range:

# Buckets by author name, finding first commit for each of them
s = create_search(source='git')

# General filters
s = add_general_date_filters(s)
s = add_bot_filter(s)
s = add_merges_filter(s)

# Filter commits to the Project Repos
s = add_project_filter(s, project_name)


# Retrieve commits from last year (2016)
s = s.filter('range', grimoire_creation_date={'gte': '2016-01-01', 'lt': '2017-01-01'})

# Bucketize by uuid and get first and last commit
s.aggs.bucket('authors', 'terms', field='author_uuid', size=100000) \
    .bucket('projects', 'terms', field='project', size=1000)
#print(s.to_dict())
result = s.execute()

df = pandas.DataFrame(columns=['author', 'project'])

for uuid in result.to_dict()['aggregations']['authors']['buckets']:
    for project in uuid['projects']['buckets']:
        df.loc[len(df)] = [uuid['key'], project['key']]

df = df.loc[(df['project'] == 'Gecko') | (df['project'] == 'Rust') | (df['project'] == 'Servo')\
            | (df['project'] == 'Firefox') | (df['project'] == 'Firefox OS (FxOS / B2G)')\
            | (df['project'] == 'WebVR')]

get_project_intersection(df)

,project,Firefox,Firefox OS (FxOS / B2G),Gecko,Rust,Servo,WebVR
0,Gecko,330.0,4.0,1211.0,80.0,329.0,7.0
1,Firefox,455.0,1.0,330.0,17.0,27.0,1.0
2,Rust,17.0,0.0,80.0,856.0,87.0,0.0
3,Servo,27.0,1.0,329.0,87.0,461.0,3.0
4,WebVR,1.0,1.0,7.0,0.0,3.0,142.0
5,Firefox OS (FxOS / B2G),1.0,13.0,4.0,0.0,1.0,1.0


## Authors across top 6 projects in 2012

In [15]:
#results = []
#for i in analyzed_range:

# Buckets by author name, finding first commit for each of them
s = create_search(source='git')

# General filters
s = add_general_date_filters(s)
s = add_bot_filter(s)
s = add_merges_filter(s)

# Filter commits to the Project Repos
s = add_project_filter(s, project_name)


# Retrieve commits from last year (2016)
s = s.filter('range', grimoire_creation_date={'gte': '2012-01-01', 'lt': '2013-01-01'})

# Bucketize by uuid and get first and last commit
s.aggs.bucket('authors', 'terms', field='author_uuid', size=100000) \
    .bucket('projects', 'terms', field='project', size=1000)
#print(s.to_dict())
result = s.execute()

df = pandas.DataFrame(columns=['author', 'project'])

for uuid in result.to_dict()['aggregations']['authors']['buckets']:
    for project in uuid['projects']['buckets']:
        df.loc[len(df)] = [uuid['key'], project['key']]

df = df.loc[(df['project'] == 'Gecko') | (df['project'] == 'Rust') | (df['project'] == 'Servo')\
            | (df['project'] == 'Firefox') | (df['project'] == 'Firefox OS (FxOS / B2G)')\
            | (df['project'] == 'WebVR')]

get_project_intersection(df)

,project,Firefox,Firefox OS (FxOS / B2G),Gecko,Rust,Servo
0,Firefox OS (FxOS / B2G),1.0,26.0,4.0,0.0,0.0
1,Gecko,252.0,4.0,673.0,24.0,19.0
2,Firefox,368.0,1.0,252.0,12.0,4.0
3,Rust,12.0,0.0,24.0,129.0,17.0
4,Servo,4.0,0.0,19.0,17.0,111.0


## Non-Employees across top 6 projects in 2016

In [16]:
#results = []
#for i in analyzed_range:

# Buckets by author name, finding first commit for each of them
s = create_search(source='git')

# General filters
s = add_general_date_filters(s)
s = add_bot_filter(s)
s = add_merges_filter(s)

# Filter commits to the Project Repos
s = add_project_filter(s, project_name)


# Retrieve commits from last year (2016)
s = s.filter('range', grimoire_creation_date={'gte': '2016-01-01', 'lt': '2017-01-01'})

# EXCLUDE EMPLOYEES
s = s.exclude('terms', author_org_name=['Mozilla Staff', 'Code Sheriff'])

# Bucketize by uuid and get first and last commit
s.aggs.bucket('authors', 'terms', field='author_uuid', size=100000) \
    .bucket('projects', 'terms', field='project', size=1000)
#print(s.to_dict())
result = s.execute()

df = pandas.DataFrame(columns=['author', 'project'])

for uuid in result.to_dict()['aggregations']['authors']['buckets']:
    for project in uuid['projects']['buckets']:
        df.loc[len(df)] = [uuid['key'], project['key']]

df = df.loc[(df['project'] == 'Gecko') | (df['project'] == 'Rust') | (df['project'] == 'Servo')\
            | (df['project'] == 'Firefox') | (df['project'] == 'Firefox OS (FxOS / B2G)')\
            | (df['project'] == 'WebVR')]

get_project_intersection(df)

,project,Firefox,Firefox OS (FxOS / B2G),Gecko,Rust,Servo,WebVR
0,Gecko,87.0,0.0,725.0,52.0,272.0,3.0
1,Firefox,177.0,0.0,87.0,2.0,5.0,1.0
2,Rust,2.0,0.0,52.0,811.0,72.0,0.0
3,Servo,5.0,0.0,272.0,72.0,398.0,2.0
4,WebVR,1.0,0.0,3.0,0.0,2.0,129.0
5,Firefox OS (FxOS / B2G),0.0,3.0,0.0,0.0,0.0,0.0


## Non-Employees across top 6 projects in 2012

In [17]:
#results = []
#for i in analyzed_range:

# Buckets by author name, finding first commit for each of them
s = create_search(source='git')

# General filters
s = add_general_date_filters(s)
s = add_bot_filter(s)
s = add_merges_filter(s)

# Filter commits to the Project Repos
s = add_project_filter(s, project_name)


# Retrieve commits from last year (2016)
s = s.filter('range', grimoire_creation_date={'gte': '2012-01-01', 'lt': '2013-01-01'})

# EXCLUDE EMPLOYEES
s = s.exclude('terms', author_org_name=['Mozilla Staff', 'Code Sheriff'])

# Bucketize by uuid and get first and last commit
s.aggs.bucket('authors', 'terms', field='author_uuid', size=100000) \
    .bucket('projects', 'terms', field='project', size=1000)
#print(s.to_dict())
result = s.execute()

df = pandas.DataFrame(columns=['author', 'project'])

for uuid in result.to_dict()['aggregations']['authors']['buckets']:
    for project in uuid['projects']['buckets']:
        df.loc[len(df)] = [uuid['key'], project['key']]

df = df.loc[(df['project'] == 'Gecko') | (df['project'] == 'Rust') | (df['project'] == 'Servo')\
            | (df['project'] == 'Firefox') | (df['project'] == 'Firefox OS (FxOS / B2G)')\
            | (df['project'] == 'WebVR')]

get_project_intersection(df)

,project,Firefox,Firefox OS (FxOS / B2G),Gecko,Rust,Servo
0,Gecko,52.0,1.0,258.0,6.0,8.0
1,Firefox,128.0,0.0,52.0,1.0,0.0
2,Servo,0.0,0.0,8.0,9.0,97.0
3,Rust,1.0,0.0,6.0,106.0,9.0
4,Firefox OS (FxOS / B2G),0.0,15.0,1.0,0.0,0.0
